# Notebook 07: Stability Analysis

## The Three Essential Stability Tests

**Before claiming a material is "stable", you MUST verify:**

1. ☑ **Thermodynamic Stability** - Formation energy, convex hull
2. ☑ **Dynamic Stability** - No imaginary phonon frequencies
3. ☑ **Mechanical Stability** - Born stability criteria satisfied

---

In [1]:
import numpy as np
from typing import Dict, List, Tuple

# Physical constants
RY_TO_EV = 13.605693122994
BOHR_TO_ANGSTROM = 0.529177210903

---

## 1. Thermodynamic Stability

### Formation Energy

For compound $A_x B_y C_z$:

$$\Delta H_f = E_{total}(A_x B_y C_z) - [x \cdot E(A) + y \cdot E(B) + z \cdot E(C)]$$

where $E(A)$, $E(B)$, $E(C)$ are energies of reference states (standard states of elements).

- **$\Delta H_f < 0$**: Stable against decomposition to elements
- **$\Delta H_f > 0$**: Metastable or unstable

In [2]:
# Reference energies (example values in eV/atom)
# These should be calculated with the SAME functional and pseudopotentials!
REFERENCE_ENERGIES_EV = {
    'Si': -5.425,    # Diamond Si
    'O':  -4.948,    # 1/2 O2 molecule (corrected)
    'Fe': -8.309,    # BCC Fe
    'Ti': -7.762,    # HCP Ti
    'Sr': -1.684,    # FCC Sr
    'Ba': -1.924,    # BCC Ba
    'Mg': -1.538,    # HCP Mg
    'Ca': -1.996,    # FCC Ca
    'Al': -3.746,    # FCC Al
}

def calculate_formation_energy(total_energy_ev: float, 
                               composition: Dict[str, int],
                               reference_energies: Dict[str, float] = REFERENCE_ENERGIES_EV) -> float:
    """
    Calculate formation energy per atom.
    
    Parameters
    ----------
    total_energy_ev : float
        Total DFT energy of compound in eV
    composition : dict
        Element counts, e.g., {'Sr': 1, 'Ti': 1, 'O': 3}
    reference_energies : dict
        Reference state energies per atom in eV
    
    Returns
    -------
    float : Formation energy in eV/atom
    """
    total_ref_energy = 0.0
    total_atoms = 0
    
    for element, count in composition.items():
        if element not in reference_energies:
            print(f"Warning: No reference for {element}")
            return None
        total_ref_energy += count * reference_energies[element]
        total_atoms += count
    
    formation_energy = (total_energy_ev - total_ref_energy) / total_atoms
    return formation_energy

# Example (hypothetical values)
print("Formation Energy Example")
print("=" * 50)
print("\nNote: Use YOUR calculated reference energies!")
print("These examples use placeholder values.")

Formation Energy Example

Note: Use YOUR calculated reference energies!
These examples use placeholder values.


### Convex Hull Analysis (CRITICAL!)

Formation energy alone is NOT sufficient!

A material can have negative formation energy but still be unstable if it decomposes into other compounds:

$$E_{hull} = E_{compound} - E_{convex\ hull}$$

- $E_{hull} = 0$: Material is ON the convex hull → **Thermodynamically stable**
- $E_{hull} > 0$: Material is ABOVE the hull → **Metastable**
- $E_{hull} < 25$ meV/atom: Potentially synthesizable

---

## 2. Mechanical Stability (Elastic Constants)

### Born Stability Criteria

A crystal is mechanically stable if and only if the elastic constant matrix is positive definite.

In [ ]:
def check_born_stability_cubic(C11: float, C12: float, C44: float) -> Tuple[bool, Dict]:
    """
    Check Born stability criteria for cubic crystals.
    
    Criteria:
    1. C11 > 0
    2. C11 - C12 > 0 (tetragonal shear stability)
    3. C11 + 2*C12 > 0 (bulk stability)
    4. C44 > 0 (shear stability)
    
    Parameters
    ----------
    C11, C12, C44 : float
        Independent elastic constants in GPa
    
    Returns
    -------
    is_stable : bool
    details : dict with individual criteria
    """
    criteria = {
        'C11 > 0': C11 > 0,
        'C11 - C12 > 0': (C11 - C12) > 0,
        'C11 + 2*C12 > 0': (C11 + 2*C12) > 0,
        'C44 > 0': C44 > 0
    }
    
    is_stable = all(criteria.values())
    
    return is_stable, criteria

def check_born_stability_hexagonal(C11: float, C12: float, C13: float, 
                                   C33: float, C44: float) -> Tuple[bool, Dict]:
    """
    Check Born stability criteria for hexagonal crystals.
    
    Criteria:
    1. C11 > |C12|
    2. C33*(C11 + C12) > 2*C13²
    3. C44 > 0
    4. C66 = (C11 - C12)/2 > 0 (automatically satisfied if criterion 1 holds)
    """
    C66 = (C11 - C12) / 2
    
    criteria = {
        'C11 > |C12|': C11 > abs(C12),
        'C33*(C11+C12) > 2*C13²': C33 * (C11 + C12) > 2 * C13**2,
        'C44 > 0': C44 > 0,
        'C66 > 0': C66 > 0
    }
    
    is_stable = all(criteria.values())
    
    return is_stable, criteria

def calculate_bulk_modulus_voigt(C11: float, C12: float, C44: float = None) -> float:
    """
    Calculate Voigt bulk modulus for cubic crystal.
    B_V = (C11 + 2*C12) / 3
    """
    return (C11 + 2 * C12) / 3

def calculate_shear_modulus_voigt(C11: float, C12: float, C44: float) -> float:
    """
    Calculate Voigt shear modulus for cubic crystal.
    G_V = (C11 - C12 + 3*C44) / 5
    """
    return (C11 - C12 + 3 * C44) / 5

# Example: Silicon elastic constants
print("Mechanical Stability Check: Silicon")
print("=" * 50)

# Experimental values for Si (GPa)
C11_Si = 166.0
C12_Si = 64.0
C44_Si = 79.6

is_stable, criteria = check_born_stability_cubic(C11_Si, C12_Si, C44_Si)

print(f"\nElastic constants (GPa):")
print(f"  C11 = {C11_Si}")
print(f"  C12 = {C12_Si}")
print(f"  C44 = {C44_Si}")

print(f"\nBorn stability criteria:")
for criterion, passed in criteria.items():
    status = '✓' if passed else '✗'
    print(f"  {status} {criterion}")

print(f"\nMechanically stable: {is_stable}")

B = calculate_bulk_modulus_voigt(C11_Si, C12_Si)
G = calculate_shear_modulus_voigt(C11_Si, C12_Si, C44_Si)
print(f"\nDerived properties:")
print(f"  Bulk modulus (Voigt): {B:.1f} GPa")
print(f"  Shear modulus (Voigt): {G:.1f} GPa")

---

## 3. Dynamic Stability (Phonons)

### What Phonons Tell Us

- **All positive frequencies**: Dynamically stable
- **Imaginary frequencies** (shown as negative): Dynamically UNSTABLE
  - Structure is at a saddle point, not a minimum
  - Must distort along the unstable mode eigenvector

### Phonon Calculation in QE

Uses Density Functional Perturbation Theory (DFPT) via `ph.x`

In [ ]:
def generate_phonon_input(prefix: str, outdir: str = './tmp',
                          fildyn: str = 'dyn', nq1: int = 2, 
                          nq2: int = 2, nq3: int = 2) -> str:
    """
    Generate ph.x input file for phonon calculation.
    
    Note: SCF must be run first with the same prefix!
    """
    input_text = f"""Phonon calculation
&INPUTPH
    prefix = '{prefix}'
    outdir = '{outdir}'
    fildyn = '{fildyn}'
    ldisp = .true.
    nq1 = {nq1}
    nq2 = {nq2}
    nq3 = {nq3}
    tr2_ph = 1.0d-14
/
"""
    return input_text

print("Phonon Calculation Input (ph.x)")
print("=" * 50)
print(generate_phonon_input('silicon', nq1=4, nq2=4, nq3=4))

print("\nWARNING: Phonon calculations are computationally expensive!")
print("For a 2x2x2 q-grid: ~8 times the cost of SCF")
print("For a 4x4x4 q-grid: ~64 times the cost of SCF")

---

## 4. Stability Checklist

### Complete Stability Assessment

```
┌─────────────────────────────────────────────────────────────┐
│                    STABILITY CHECKLIST                      │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  □ THERMODYNAMIC STABILITY                                  │
│    □ Formation energy calculated                           │
│    □ ΔHf < 0 (stable vs elements)                          │
│    □ Ehull < 25 meV/atom (synthesizable)                   │
│                                                             │
│  □ DYNAMIC STABILITY                                        │
│    □ Phonon calculation completed                          │
│    □ No imaginary frequencies                              │
│    □ Acoustic modes go to zero at Γ                        │
│                                                             │
│  □ MECHANICAL STABILITY                                     │
│    □ Elastic constants calculated                          │
│    □ All Born criteria satisfied                           │
│    □ Bulk modulus > 0                                      │
│                                                             │
│  ═══════════════════════════════════════════════════════   │
│  ALL CHECKS PASSED? → Proceed to property calculations     │
│  ANY CHECK FAILED?  → Re-examine structure or conclude     │
│                       material is unstable                 │
└─────────────────────────────────────────────────────────────┘
```

---

## Summary

### Key Points

1. **Never skip stability tests** - An unstable material's properties are meaningless
2. **Thermodynamic**: Check formation energy AND convex hull distance
3. **Dynamic**: No imaginary phonon frequencies allowed
4. **Mechanical**: Born stability criteria must be satisfied

### What If Tests Fail?

| Test | Failure | Action |
|------|---------|--------|
| Thermodynamic | High Ehull | Material may not be synthesizable |
| Dynamic | Imaginary modes | Distort structure along mode eigenvector |
| Mechanical | Born criteria | Material is mechanically unstable |

### Next Notebook
→ **08_Electronic_Properties.ipynb**: Calculate band structure and DOS